<font size=6><b>**Я.Практикум. Проект №13**</b></font>
    
<font size=6><b>**Проект для «Викишоп» с BERT**</b></font>

***

# Постановка задачи

**Заказчик**  
Интернет-магазин «Викишоп»

**Цель**  
Разработать модель машинного обучения, которая будет искать токсичные комментарии и отправлять их на модерацию.
В качестве метрики качества следует использовать F1, значение которой должно быть не менее 0,75.

**Задачи**  

1. Загрузить и исследовать данные;
1. Подготовить три вида признаков:
    - очищенный незакодированный текст;
    - TF-IDF;
    - emedings при помощи модели BERT; 
1. Обучить на полученных признаках модель логистической регрессии (LR);
1. Обучить на очищенном тексте модель CatBoost (CB) классифицировать позитивные и негативные комментарии на разных признаках;
1. Используя метрику ROC-AUC определить лучшее сочетание модели и признаков;
1. Для лучшей модели выбрать оптимальный порог (threshold) по критерию максимизации значения метрики F1;  
1. Сделать выводы.

**Исходные данные**  
Набор данных с разметкой о токсичности правок.

# Подготовка окружения

Загрузим библиотеки, необходимые для выполнения проекта

In [1]:
import numpy as np
import pandas as pd
import nltk
import re

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    StratifiedKFold,
)
from tqdm.notebook import tqdm

try:
    from catboost import CatBoostClassifier, Pool, cv
except:
    !pip install catboost
    from catboost import CatBoostClassifier, Pool, cv

try:
    import torch
except:
    !pip install torch
    import torch

try:
    import transformers
except:
    !pip install transformers
    import transformers


# автоформатирование
%load_ext lab_black

nltk.download("punkt")
nltk.download("stopwords")

Загрузим сами данные.

In [2]:
try:
    df = pd.read_csv("D:/jupyter/practicum_13/toxic_comments.csv")
except:
    df = pd.read_csv("https://code.s3.yandex.net/datasets/toxic_comments.csv")

# Обзор данных

Вызовем первые 5 строк, общую информацию о таблице, а также количество дубликатов.

In [3]:
df.head()

In [4]:
df.info()

In [5]:
df.duplicated().sum()

Датафрейм содержит 159 571 объекта без пропусков и явных повторений.
Каждый объект включает текст комментария `text` и целевой признак `toxic`.
При этом, комментарии сделаны на английском языке.

Проверим дисбаланс классов.

In [6]:
df["toxic"].mean()

Только 10% данных содержат токсичные комментарии.
Такой дисбаланс должен быть учтен при разбиении датафрема на обучающую и тестовую выборки.

Рассмотрим детально первую строку с текстом.

In [7]:
df["text"][0]

Текстовая информация содержит следующие особенности:  
* используется различный регистр;
* используются скрытые текстовые символы (\n, \t...);
* используются числовые символы;
* используется различная форма слов.

Для улучшения значения целевой метрики при подготовке признаков перечисленные недостатки целесообразно устранить.

# Подготовка признаков

## Очистка текста

Проведем обработку строк: очистим их от лишних символов и стоп-слов.

In [8]:
stopwords = set(nltk_stopwords.words("english"))

In [9]:
def clear(raw):
    text = raw["text"].lower()
    text = " ".join(re.sub(r"[^a-zA-Z ]", " ", text).split())
    text = " ".join([w for w in word_tokenize(text) if not w in stopwords])
    return text

In [10]:
df["text"] = df.apply(clear, axis=1)

Проверим выполнение операции

In [11]:
df["text"][0]

Очистка выполнена.
Однако при этом могли проявится дубликаты в данных, которые были неявными при первоначальном обзоре.

In [12]:
df.duplicated().sum()

Количество таких дубликатов составляет около 1%, в результате чего они могут быть безболезненно удалены.

In [13]:
df = df.drop_duplicates().reset_index(drop=True)

In [14]:
df.duplicated().sum()

Признак с очищенным текстом сформирован, можно переходить к следующему шагу.

## Stemming

Выполним stemming по алгоритму Портера.
Полученный указанной обработкой корпус слов сохраним в соответствующий столбец.

Для повышения стабильности обработки добавим заполнение корпуса пропуском в случае, если не удается выполнить stemming.

In [15]:
porter = PorterStemmer()

In [16]:
def porterstem(raw):
    text = raw["text"]
    try:
        text = " ".join([porter.stem(word) for word in text.split()])
    except:
        text = np.NaN
    return text

In [17]:
def encoder(feature_name, batch_size, function_name):
    df[feature_name] = np.NaN
    for i in tqdm(range(df.shape[0] // batch_size)):
        df[feature_name][batch_size * i : batch_size * (i + 1)] = df[
            batch_size * i : batch_size * (i + 1)
        ].apply(function_name, axis=1)

    df[feature_name][batch_size * (i + 1) :] = df[batch_size * (i + 1) :].apply(
        function_name, axis=1
    )

In [18]:
encoder("corpus", 10000, porterstem)

Проверим количество пропусков.

In [19]:
df.isna().sum()

Один объект не обработался. 
Выведем строку с ним и содержимое ячейки с комментарием.

In [20]:
df.loc[df["corpus"].isna()]

In [21]:
df.loc[df["corpus"].isna()]["text"].values

По всей видимости, объект - шутка разработчиков задачи.
Учитывая, что он не несет смысловой нагрузки, исключим его из рассмотрения. 

In [22]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

Полученный stemming'ом корпус слов используем при определении TF-IDF.
Учитывая, что указанный параметр зависит от состава выборки, его определение целесообразно проводить только после разделения датафрейма на обучающую, валидационную и тестовую выборки.

## Embeding

Инициализируем токенизатор и модель BERT на основе базовой предобученной из стандартной библиотеки.

In [23]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model_bert = transformers.DistilBertModel.from_pretrained("distilbert-base-uncased")

Преобразуем текст в номера токенов из словаря методом encode.
Учитывая, что выбранная модель BERT предобучена на текстах длинной 512 слов, добавим соответствующий параметр в метод encode.

In [42]:
def bert_token(raw):
    text = raw["text"]
    try:
        text = tokenizer.encode(text, add_special_tokens=True, max_length=512)
    except:
        text = np.NaN
    return text

In [43]:
encoder("corpus_bert", 10000, bert_token)

Проверим количество пропущенных объектов.

In [45]:
df.isna().sum()

Для всех объектов успешно созданы токены.

Применим метод padding, чтобы уравнять длины исходных текстов в корпусе. 

In [49]:
padded = np.array([i + [0] * (512 - len(i)) for i in df["corpus_bert"].values])

Отбросим нулевые токены и создадим attention_mask для действительно важных из них. 

In [50]:
attention_mask = np.where(padded != 0, 1, 0)

Определим сами embedings.

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def batch_embeder(low_step, high_step):
    # преобразуем данные
    batch = torch.tensor(padded[low_step:high_step]).to(device)
    # преобразуем маску
    attention_mask_batch = torch.tensor(attention_mask[low_step:high_step]).to(device)

    with torch.no_grad():
        model_bert.to(device)
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)

    # преобразуем элементы методом numpy() к типу numpy.array
    return batch_embeddings[0][:, 0, :].cpu().numpy()

In [59]:
embeddings = []
batch_size = 50

for i in tqdm(range(padded.shape[0] // batch_size)):
    embeddings.append(batch_embeder(batch_size * i, batch_size * (i + 1)))

# финальный проход
embeddings.append(batch_embeder(batch_size * (i + 1), None))

Соберём все embedings в матрицу признаков вызовом функции concatenate():

In [60]:
features_bert = np.concatenate(embeddings)

Embedings созданы.
Можно переходить к разделению датафрейма на подвыборки.

## Разделение на подвыборки

Разделим данные на подвыборки в пропорции 3:1:1 с сохранением дисбаланса классов.

In [61]:
train_features_bert, val_test_features_bert, train, val_test = train_test_split(
    features_bert, df, test_size=0.4, random_state=26, stratify=df["toxic"]
)

val_features_bert, test_features_bert, val, test = train_test_split(
    val_test_features_bert,
    val_test,
    test_size=0.5,
    random_state=26,
    stratify=val_test["toxic"],
)

In [62]:
train["toxic"].mean(), val["toxic"].mean(), test["toxic"].mean()

Дисбаланс сохранен.

Теперь необходимо сформировать значения TF-IDF корпуса текстов.
Для выполнения вычислений используем соответствующую библиотеку. 

In [63]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

train_tf_idf = count_tf_idf.fit_transform(train["corpus"])
val_tf_idf = count_tf_idf.transform(val["corpus"])
test_tf_idf = count_tf_idf.transform(test["corpus"])

Признаки TF-IDF сформированы, можно приступать к моделированию.

# Моделирование

## Логистическая регрессия

Определим модель и проведем ее обучение на двух типах признаков: TF-IDF и BERT embedings.

Одним из ключевых гиперпараметров LR является вес классов.
В этой связи проведем его подбор при помощи метода случайного поиска.

In [64]:
parameters = {"C": np.linspace(0.0001, 100, 20)}
skf = StratifiedKFold(5, shuffle=True, random_state=26)

In [71]:
model_lr_tf_idf = RandomizedSearchCV(
    estimator=LogisticRegression(max_iter=500, random_state=26, n_jobs=-1),
    param_distributions=parameters,
    n_jobs=-1,
    n_iter=5,
    random_state=26,
)
model_lr_tf_idf.fit(train_tf_idf, train["toxic"])

In [72]:
model_lr_bert = RandomizedSearchCV(
    estimator=LogisticRegression(max_iter=500, random_state=26, n_jobs=-1),
    param_distributions=parameters,
    n_jobs=-1,
    cv=skf,
    n_iter=5,
    random_state=26,
)
model_lr_bert.fit(train_features_bert, train["toxic"])

Определим значение метрики ROC-AUC на валидационной подвыборке.
Указанная метрика не зависит от порога для предсказания класса, в связи с чем ее целесообразно использовать для сравнения разных моделей и определения лучшей из них.

In [74]:
roc_auc_score(val["toxic"], model_lr_tf_idf.predict(val_tf_idf))

In [76]:
roc_auc_score(val["toxic"], model_lr_bert.predict(val_features_bert))

По полученным значениям видно, что обучение LR на признаках TF-IDF на 3,7% эффективнее обучения на embedings.

Проверим качество обучение модели CB.

## Catboost

Модель CB имеет встроенные механизмы формирования embedings.
В этой связи ей достаточно передать только очищенный текст, обозначив его соответствующим образом.

In [94]:
train_pool = Pool(
    data=train[["corpus"]],
    label=train["toxic"],
    feature_names=["corpus"],
    text_features=["corpus"],
)

val_pool = Pool(
    data=val[["corpus"]],
    label=val["toxic"],
    feature_names=["corpus"],
    text_features=["corpus"],
)

Обучение модели СВ проведем при помощи встроенного метода кросс-валидации.

In [81]:
%%time 
params = {
    "eval_metric": "Logloss",
    "loss_function": "Logloss",
    "learning_rate": 0.1,
    "random_seed": 26,
    "verbose": 250,
    'early_stopping_rounds':200,
}

cv_data = cv(
    pool=train_pool,
    params=params,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=True,
    verbose=False,
    early_stopping_rounds=200,
    return_models=True,
    plot=True,
)

Лучшая модель получена на фолде с номером 3.
Определим для нее метрику ROC-AUC.

In [82]:
roc_auc_score(val["toxic"], cv_data[1][3].predict(val_pool, prediction_type="Class"))

Полученное значение на 0,7% хуже соответствующего, полученного при обучении модели LR на признаках TF-IDF.
В этой связи, для полноценного сравнения проведем аналогичное обучение на тех же признаках.

In [83]:
train_pool_tf_idf = Pool(
    data=train_tf_idf,
    label=train["toxic"],
)

val_pool_tf_idf = Pool(
    data=val_tf_idf,
    label=val["toxic"],
)

In [86]:
%%time 
cv_data_tf_idf = cv(
    pool=train_pool_tf_idf,
    params=params,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=True,
    verbose=False,
    early_stopping_rounds=200,
    return_models=True,
    plot=True,
)

In [87]:
roc_auc_score(
    val["toxic"], cv_data_tf_idf[1][0].predict(val_pool_tf_idf, prediction_type="Class")
)

Полученное значение хуже полученного при кодировании текста внутренними средствами модели CB.

Таким образом, по метрике ROC-AUC на валидационной выборке лучшей из рассмотренных моделей, является модель LR, обученная на признаках TF-IDF.

На следующем шаге определим лучшее значение порога для расчета целевой метрики F1.

## Выбор порога классификации

Определим значение целевой метрики со стандартным порогом, равным 0,5. 

In [88]:
f1_score(val["toxic"], model_lr_tf_idf.predict(val_tf_idf))

Полученное значение выше заданного по условиям задачи критерия 0,75. 
Учитывая, что на тестовой выборке указанная метрика, скорее всего, изменится в худшую сторону, целесообразно провести анализ  возможности ее улучшения за счет изменения порога классификации.
Для этого создадим столбец со значениями вероятности определения к тому или иному классу.

In [89]:
val["pred_lr_tf_idf_proba"] = np.NaN
val["pred_lr_tf_idf_proba"] = 1 - model_lr_tf_idf.predict_proba(val_tf_idf)

Определим перечень всех существующих значений вероятностей и добавим его к нулевому значению.

In [90]:
thrs = [0] + list(val["pred_lr_tf_idf_proba"].unique())

Переберем в цикле значения метрики F1 при изменении порога по всем значениям вероятности.

In [91]:
result = []

for thr in tqdm(thrs):
    val["pred_lr_tf_idf_best"] = (val["pred_lr_tf_idf_proba"] > thr) * 1
    result.append((thr, f1_score(val["toxic"], val["pred_lr_tf_idf_best"])))

Выведем значение порога, обеспечивающего получение максимальное значение целевой метрики. 

In [92]:
t = pd.DataFrame(result, columns=["thr", "f1"])

thr_best = t.loc[t["f1"] == t["f1"].max(), "thr"].values[0]

t[t["f1"] == t["f1"].max()]

Полученное значение лучше "ненастроенного" на 0,5%.

В дальнейшем будем использовать его для расчета целевой метрики на тестовой подвыборке.

## Тестирование лучшей модели

Проведем расчет целевой метрики F1 на тестовой подвыборке.

In [93]:
test["pred_lr_tf_idf_proba"] = np.NaN
test["pred_lr_tf_idf_proba"] = 1 - model_lr_tf_idf.predict_proba(test_tf_idf)
test["pred_lr_tf_idf_best"] = (test["pred_lr_tf_idf_proba"] > thr_best) * 1
f1_score(test["toxic"], test["pred_lr_tf_idf_best"])

Полученное значение выше заданного в условии задания на 2,5%.

# Выводы

Для достижения поставленной условием задания цели решены следующие задачи:
1. Загружены и исследованы данные;
1. Проведена очистка незакодированного текста от лишних символов и стоп-слов;
1. Определены признаки TF-IDF;
1. Сформированы emedings при помощи предобученной модели BERT из стандартной библиотеки "distilbert-base-uncased"; 
1. Обучены на полученных признаках модели логистической регрессии (LR) и CatBoost (CB);
1. Использована метрика ROC-AUC для определения лучшего сочетание модели и признаков. 
Получено, что для обученной на признаках TF-IDF модели LR указанная метрика имеет наивысшее значение, равное 84,5%;
1. Для указанной модели выбран порог, обеспечивающий максимизацию значения метрики F1. 

В результате работы разработана модель LR, определяющая токсичные комментарии, со значением метрики F1, равным 77,5%, что лучше заданного в условии порога на 2,5%.
Разработанная модель может быть использована для отправки токсичных комментариев на модерацию.